## Problem *G3*

![priklad](../obrazky/priklad_g3.jpg)

## Solution:

### State 1
- Saturated Vapor

In [23]:
from pyfluids import Fluid, Input, FluidsList

m_flow = 0.05 # kg/s
x1 = 1.0
p1 = 140_000 # Pa
p2 = 800_000 # Pa

Refridgerant = Fluid(FluidsList.R134a)
state1 = Refridgerant.with_state(
    Input.pressure(p1),
    Input.quality(x1)
)

### State 2

- Isoentropic compression   


In [24]:
state2 = state1.isentropic_compression_to_pressure(p2)
w12 = state2.enthalpy - state1.enthalpy

### State 3
- Saturated liquid

In [25]:
state3 = Refridgerant.with_state(
    Input.quality(0),
    Input.pressure(p2)
)

# Heat rejection (cooling)
q23 = state3.enthalpy - state2.enthalpy
Q23 = m_flow * q23

### State 3-4:
- Throttling
- constant enthalpy


In [26]:
state4 = state3.isenthalpic_expansion_to_pressure(p1)

## Results:

In [31]:
# Heat recived from the fridge compartment
q41 = state1.enthalpy - state4.enthalpy
Q41 = m_flow * q41

w_o = q41 - abs(q23)
W_komp = m_flow * w_o

e_ch = q41 / abs(w_o)
e_t = q23 / w_o
s_prod34 = state4.entropy - state3.entropy

print(f'Compressor power: {W_komp:.2f} W')
print(f'Heat rejection: {Q23:.2f} W')
print(f'Heat addition from fridge compartment: {Q41:.2f} W')
print(f'Thermal efficiency: {e_ch:.3f}')


Compressor power: -1810.60 W
Heat rejection: -8994.33 W
Heat addition from fridge compartment: 7183.73 W
Thermal efficiency: 3.968


## T-S Diagram of Rankin Cycle

In [28]:
import plotly.graph_objects as go
import numpy as np

state2_1 = state1.dew_point_at_pressure(state2.pressure)

states = [state1, state2, state2_1, state3, state4, state1]

T = [state.temperature for state in states]
S = [state.entropy for state in states]

# Plot the saturation lines
t_range = np.linspace(Refridgerant.triple_temperature, Refridgerant.critical_temperature, 1000)
S_liq = [Refridgerant.bubble_point_at_temperature(t).entropy for t in t_range]
S_vap = [Refridgerant.dew_point_at_temperature(t).entropy for t in t_range]

# Create a trace for liquid saturation line
trace0 = go.Scatter(x=S_liq, y=t_range, mode="lines", name="Liquid Saturation Line")

# Create a trace for vapor saturation line
trace1 = go.Scatter(x=S_vap, y=t_range, mode="lines", name="Vapor Saturation Line")

# Create a trace for the cycle
trace2 = go.Scatter(x=S, y=T, mode="lines+markers", name="Cycle")

data = [trace0, trace1, trace2]

layout = go.Layout(
    title="T-S Diagram",
    xaxis=dict(title="Entropy (J/kg*K)"),
    yaxis=dict(title="Temperature (K)"),
)

fig = go.Figure(data=data, layout=layout)

fig.show()